# 🏥 고양이 의료 문서 분석

이 노트북은 고양이 혈액검사 파일을 멀티모달 LLM 으로 분석하여 데이터를 추출합니다.
(현재 GPT-4V)

## 주요 기능
- PDF → 이미지 변환
- 멀티모달 LLM을 통한 테이블 데이터 추출
- 의료 검사 결과 구조화
- 검사결과 분석

## 1. 환경 설정 및 라이브러리 로드

In [ ]:
# 필요한 라이브러리 설치
!pip install pytesseract pymupdf

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI

# 환경변수 로드
load_dotenv("/home/aidan/work/kosa-ict-genai-2025-2nd/src/exercise/aidanpark/.env")

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print(f"🔑 API Key 설정: {'✅ 완료' if os.getenv('OPENAI_API_KEY') else '❌ 없음'}")

## 2. 업로드 파일 전처리

In [ ]:
import io
from re import T
from typing import List, Dict, Any
import os
import fitz
import tempfile
from PIL import Image, ImageFilter, ImageOps
from typing import Callable, Sequence

import numpy as np
from pytesseract import image_to_data, Output
import pytesseract

def pdf_to_images(pdf_path: str, resolution_scale: float = 1.5) -> List[bytes]:
    """
    PDF 파일 경로를 받아 각 페이지를 PNG 바이트로 변환해 리스트로 반환
    """
    png_list: List[bytes] = []
    doc = None
    try:
        doc = fitz.open(pdf_path)
        mat = fitz.Matrix(resolution_scale, resolution_scale)
        for page in doc:
            pix = page.get_pixmap(matrix=mat, alpha=False)
            png_list.append(pix.tobytes("png"))
    finally:
        try:
            if doc is not None:
                doc.close()
        except Exception:
            pass
    return png_list

def open_with_exif(img_bytes: bytes):
    """
    1) 로드 + EXIF 회전 교정: 카메라 회전 정보가 있으면 실제 픽셀을 회전
    """
    img = Image.open(io.BytesIO(img_bytes))
    try:
        img = ImageOps.exif_transpose(img)
    except Exception:
        pass
    return img

def flatten_transparency(img: Image.Image) -> Image.Image:
    """
    2) 투명 채널 플래튼: RGBA/LA → 흰 배경 위에 합성 (기호/점 주변 헤일로 방지)
    """
    if img.mode in ("RGBA", "LA"):
        bg = Image.new("RGB", img.size, (255, 255, 255))
        alpha = img.split()[-1]
        return Image.composite(img.convert("RGB"), bg, alpha)
    return img

def auto_crop_with_margin(img: Image.Image, margin: int = 10) -> Image.Image:
    """
    3) 자동 크롭: 흰 배경을 기준으로 내용물 bbox를 찾고 margin만큼 여유
    """
    gray = img.convert("L")
    inv = ImageOps.invert(gray)  # 흰색 배경 → 0, 잉크 → 양수
    bbox = inv.getbbox()
    if not bbox:
        return img
    x0, y0, x1, y1 = bbox
    x0 = max(0, x0 - margin)
    y0 = max(0, y0 - margin)
    x1 = min(img.width,  x1 + margin)
    y1 = min(img.height, y1 + margin)
    return img.crop((x0, y0, x1, y1))

def upscale_min_resolution(img: Image.Image, min_long_edge: int = 1200) -> Image.Image:
    """
    4) 해상도 표준화(업스케일만): 소수점/기호 선명도를 위해 최소 해상도 확보, 과대 크기는 축소하지 않음
    """
    w, h = img.size
    long_edge = max(w, h)
    if long_edge < min_long_edge:
        scale = min_long_edge / float(long_edge)
        img = img.resize((int(round(w * scale)), int(round(h * scale))), Image.LANCZOS)
    return img

def normalize_mode(img: Image.Image) -> Image.Image:
    """
    5) 모드 정규화: 이후 연산 호환을 위해 L 또는 RGB로 제한
    """
    if img.mode not in ("L", "RGB"):
        return img.convert("RGB")
    return img

def weak_autocontrast(img: Image.Image, cutoff: float = 0.4) -> Image.Image:
    """
    6) 자동 대비: 낮은 컷오프(0.4%)로 미세 픽셀(소수점·단위) 클리핑 방지
    """
    return ImageOps.autocontrast(img, cutoff=cutoff)

def conservative_sharpen(img: Image.Image, radius: float = 1.0, percent: int = 120, threshold: int = 3, enabled: bool = True) -> Image.Image:
    """
    7) 보수적 샤픈: 소수점 주변 헤일로 없이 스트로크만 강화
    """
    if not enabled:
        return img
    return img.filter(ImageFilter.UnsharpMask(radius=radius, percent=percent, threshold=threshold))

def to_grayscale(img: Image.Image, enabled: bool = True) -> Image.Image:
    """
    8) 그레이스케일: 문서형 이미지에서 텍스트 대비를 높여 인식 안정화
    """
    if enabled and img.mode != "L":
        return img.convert("L")
    return img

def table_smart_crop(
    img: Image.Image,
    enabled: bool = False,
    lang: str = "eng+kor",
    conf_min: int = 30,
    top_margin: int = 18,
    bottom_margin: int = 10,
    min_header_hits: int = 2,
    first_col_tolerance: int = 80,
    gap_multiplier: float = 2.8,
    min_rows: int = 5,
    debug: bool = True,
    min_table_height: int = 100,  # 🆕 최소 테이블 높이 파라미터
) -> Image.Image:
    """
    테이블 스마트 크롭(1순위: Tesseract TSV 앵커)
    - 상단: 헤더(국/영 키워드) 또는 바이오마커 열의 첫 클러스터 시작
    - 하단: 다음 헤더 직전 또는 큰 수직 공백(행간 중앙값*k) 직전
    - 여러 테이블이 있어도 가장 위 테이블 1개만 남김
    """
    if not enabled:
        if debug:
            print("[table_smart_crop] ⏭️ 비활성화 상태 - 원본 반환")
        return img
    
    if debug:
        print(f"[table_smart_crop] 🚀 시작 - 이미지 크기: {img.size}")
    
    try:
        pil = img if img.mode in ("L", "RGB") else img.convert("RGB")
        if debug:
            print(f"[table_smart_crop] 🖼️ 이미지 모드: {pil.mode}")

        # OCR
        if debug:
            print("[table_smart_crop] 📝 OCR 시작 (Tesseract)...")
        config = "--oem 3 --psm 4"
        data = image_to_data(pil, lang=lang, config=config, output_type=Output.DICT)
        n = len(data.get("text", []))
        if debug:
            print(f"[table_smart_crop] 📊 OCR 완료 - 총 {n}개 텍스트 요소 감지")
        
        if n == 0:
            if debug:
                print("[table_smart_crop] ⚠️ OCR 결과 없음 - 원본 반환")
            return img

        # 단어 토큰 필터링
        if debug:
            print(f"[table_smart_crop] 🔍 토큰 필터링 (최소 신뢰도: {conf_min})...")
        tokens = []
        for i in range(n):
            txt = (data["text"][i] or "").strip()
            if not txt:
                continue
            try:
                conf = float(data["conf"][i])
            except Exception:
                conf = -1
            if conf < conf_min:
                continue
            tokens.append(
                {
                    "text": txt,
                    "x": int(data["left"][i]),
                    "y": int(data["top"][i]),
                    "w": int(data["width"][i]),
                    "h": int(data["height"][i]),
                    "page": int(data["page_num"][i]),
                    "block": int(data["block_num"][i]),
                    "par": int(data["par_num"][i]),
                    "line": int(data["line_num"][i]),
                }
            )
        
        if debug:
            print(f"[table_smart_crop] ✅ 유효한 토큰: {len(tokens)}개")
        
        if not tokens:
            if debug:
                print("[table_smart_crop] ⚠️ 유효한 토큰 없음 - 원본 반환")
            return img

        # 라인 그룹핑
        if debug:
            print("[table_smart_crop] 📋 라인 그룹핑 시작...")
        from collections import defaultdict
        groups = defaultdict(list)
        for t in tokens:
            key = (t["page"], t["block"], t["par"], t["line"])
            groups[key].append(t)

        lines = []
        for key, group in groups.items():
            x0 = min(g["x"] for g in group)
            y0 = min(g["y"] for g in group)
            x1 = max(g["x"] + g["w"] for g in group)
            y1 = max(g["y"] + g["h"] for g in group)
            text_norm = " ".join(g["text"] for g in group).lower()
            lines.append({"x0": x0, "y0": y0, "x1": x1, "y1": y1, "text": text_norm})
        
        if debug:
            print(f"[table_smart_crop] ✅ 라인 그룹핑 완료: {len(lines)}개 라인")
        
        if not lines:
            if debug:
                print("[table_smart_crop] ⚠️ 라인 없음 - 원본 반환")
            return img
        
        lines.sort(key=lambda l: l["y0"])

        # 헤더 후보 판단(국/영 혼합)
        if debug:
            print("[table_smart_crop] 🔎 헤더 라인 검색 중...")
        header_kws = {
            "name", "unit", "result", "reference", "ref", "min", "max",
            "항목", "검사항목", "결과", "단위", "기준치", "참조치", "참고치"
        }
        def is_header(l) -> bool:
            hits = sum(1 for kw in header_kws if kw in l["text"])
            return hits >= min_header_hits and len(l["text"]) >= 4

        header_lines = [l for l in lines if is_header(l)]
        header_lines.sort(key=lambda l: l["y0"])
        
        if debug:
            print(f"[table_smart_crop] 📌 헤더 라인 발견: {len(header_lines)}개")
            if header_lines:
                for idx, h in enumerate(header_lines[:5]):  # 🆕 최대 5개만 출력
                    print(f"[table_smart_crop]    헤더 {idx+1}: y={h['y0']}, text='{h['text'][:50]}...'")

        # 바이오마커 앵커(헤더 실패 대비)
        if debug:
            print("[table_smart_crop] 🧬 바이오마커 검색 중...")
        biomarker_kws = {
            "rbc","hct","hgb","mcv","mch","mchc","plt","wbc",
            "retic","glucose","glu","bun","crea","creatinine","ast","alt","alp",
            "mpv","pct","rdw","neut","lymph","mono","eos","baso"
        }
        def has_biomarker(l) -> bool:
            t = l["text"].replace("−", "-")
            return any(kw in t for kw in biomarker_kws)

        # 상단 y 결정
        if header_lines:
            if debug:
                print("[table_smart_crop] ✅ 헤더 기반으로 상단 계산")
            top_header = header_lines[0]
            y_top = max(0, top_header["y0"] - top_margin)
            anchor_x = top_header["x0"]
            if debug:
                print(f"[table_smart_crop]    y_top={y_top}, anchor_x={anchor_x}")
        else:
            if debug:
                print("[table_smart_crop] 🔄 헤더 없음 - 바이오마커 앵커 사용")
            biomarker_lines = [l for l in lines if has_biomarker(l)]
            if not biomarker_lines:
                if debug:
                    print("[table_smart_crop] ❌ 바이오마커도 없음 - 원본 반환")
                return img
            
            if debug:
                print(f"[table_smart_crop] ✅ 바이오마커 라인: {len(biomarker_lines)}개")
            
            biomarker_lines.sort(key=lambda l: l["y0"])
            xs = np.array([l["x0"] for l in biomarker_lines])
            anchor_x = int(np.percentile(xs, 20))
            first_band = [l for l in biomarker_lines if abs(l["x0"] - anchor_x) <= first_col_tolerance] or biomarker_lines[:1]
            y_top = max(0, min(l["y0"] for l in first_band) - top_margin)
            if debug:
                print(f"[table_smart_crop]    y_top={y_top}, anchor_x={anchor_x}")

        # 🆕 하단 y 결정 (개선된 로직)
        if debug:
            print("[table_smart_crop] 📐 하단 경계 계산 중...")
        
        # 다음 헤더 필터링: 충분히 멀리 떨어진 헤더만 고려
        below_headers = [l for l in header_lines if l["y0"] > y_top + min_table_height]
        
        if debug:
            print(f"[table_smart_crop]    충분히 멀리 떨어진 헤더: {len(below_headers)}개 (최소 거리: {min_table_height}px)")
        
        if below_headers:
            # 다음 테이블의 헤더를 찾았다면 그 직전까지
            y_bottom = max(y_top + min_table_height, below_headers[0]["y0"] - 10)
            if debug:
                print(f"[table_smart_crop] ✅ 다음 테이블 헤더로 하단 결정: y_bottom={y_bottom}")
        else:
            # 다음 헤더가 없으면 공백 분석으로 하단 결정
            if debug:
                print("[table_smart_crop] 🔄 다음 헤더 없음 - 공백 분석으로 하단 결정")
            candidate = [l for l in lines if l["y0"] >= y_top]
            band = [l for l in candidate if abs(l["x0"] - anchor_x) <= first_col_tolerance]
            band.sort(key=lambda l: l["y0"])
            
            if debug:
                print(f"[table_smart_crop]    테이블 후보 라인: {len(band)}개")
            
            if len(band) < min_rows:
                if debug:
                    print(f"[table_smart_crop] ⚠️ 최소 행 수 부족 ({len(band)} < {min_rows})")
                    print(f"[table_smart_crop] 🔄 전체 라인에서 공백 분석 시도...")
                # 🆕 첫 열 제약 완화: 전체 라인에서 y_top 이후 모든 라인 고려
                band = [l for l in lines if l["y0"] >= y_top]
                band.sort(key=lambda l: l["y0"])
                
                if len(band) < min_rows:
                    if debug:
                        print(f"[table_smart_crop] ❌ 전체 라인도 부족 ({len(band)} < {min_rows}) - 원본 반환")
                    return img
            
            gaps = [band[i+1]["y0"] - band[i]["y1"] for i in range(len(band)-1)]
            med_gap = np.median([g for g in gaps if g >= 0]) if gaps else 20
            thresh = max(24, med_gap * gap_multiplier)
            
            if debug:
                print(f"[table_smart_crop]    행간 중앙값: {med_gap:.1f}px, 공백 임계값: {thresh:.1f}px")

            y_bottom = band[0]["y1"]
            prev_y1 = band[0]["y1"]
            row_count = 1
            for l in band[1:]:
                if (l["y0"] - prev_y1) > thresh:
                    if debug:
                        print(f"[table_smart_crop]    큰 공백 감지: {l['y0'] - prev_y1:.1f}px > {thresh:.1f}px")
                    break
                y_bottom = max(y_bottom, l["y1"])
                prev_y1 = l["y1"]
                row_count += 1
            
            y_bottom = min(img.height, int(y_bottom + bottom_margin))
            if debug:
                print(f"[table_smart_crop] ✅ 하단 결정: y_bottom={y_bottom}, 테이블 행 수={row_count}")

        # 최종 검증
        final_height = y_bottom - y_top
        if final_height < 64:
            if debug:
                print(f"[table_smart_crop] ❌ 테이블 높이 부족 ({final_height}px < 64px) - 원본 반환")
            return img

        # 크롭 실행
        cropped = img.crop((0, int(y_top), img.width, int(y_bottom)))
        if debug:
            print(f"[table_smart_crop] ✂️ 크롭 완료")
            print(f"[table_smart_crop]    원본: {img.size} → 크롭: {cropped.size}")
            print(f"[table_smart_crop]    영역: y_top={int(y_top)}, y_bottom={int(y_bottom)}")
            print(f"[table_smart_crop]    크롭 높이: {final_height}px")
        
        return cropped

    except Exception as e:
        if debug:
            print(f"[table_smart_crop] ❌ 오류 발생: {e}")
            import traceback
            traceback.print_exc()
        return img

def add_white_border(img: Image.Image, border: int = 4) -> Image.Image:
    """
    10) 테두리 추가: 가장자리 문자가 잘리지 않도록 얇은 흰색 여백
    """
    if border and border > 0:
        fill = 255 if img.mode == "L" else (255, 255, 255)
        return ImageOps.expand(img, border=border, fill=fill)
    return img

def downscale_target_long_edge(img: Image.Image, target_long_edge: int = 1800) -> Image.Image:
    """
    11) 해상도 표준화(다운스케일 허용): 과대 크기는 축소
    """
    w, h = img.size
    long_edge = max(w, h)
    if long_edge > target_long_edge:
        scale = target_long_edge / float(long_edge)
        img = img.resize((int(round(w * scale)), int(round(h * scale))), Image.LANCZOS)
    return img

def save_png_bytes(img: Image.Image, compress_level: int = 6) -> bytes:
    """
    12) PNG 저장(무손실): 텍스트/기호 보존에 유리
    """
    buf = io.BytesIO()
    img.save(buf, format="PNG", optimize=True, compress_level=compress_level)
    return buf.getvalue()

def apply_pipeline(img: Image.Image, steps: Sequence[tuple[Callable, dict]]) -> Image.Image:
    """
    체이닝 실행 유틸. [(func, kwargs), ...] 형태로 전달된 스텝을 순서대로 적용.
    """
    for func, kwargs in steps:
        img = func(img, **kwargs)
    return img

def preprocess_image_bytes(
    img_bytes: bytes,
    # 파라미터 (튜닝 편의)
    min_long_edge: int = 1200,
    target_long_edge: int = 1800,
    crop_margin: int = 10,
    autocontrast_cutoff: float = 0.4,
    sharpen_radius: float = 1.0,
    sharpen_percent: int = 120,
    sharpen_threshold: int = 3,
    grayscale: bool = True,
    add_border: int = 4,
    png_compress_level: int = 6,
    # 스위치(각 스텝 온/오프)
    do_exif: bool = True,
    do_flatten_alpha: bool = True,
    do_autocrop: bool = True,
    do_upscale_min: bool = True,
    do_normalize_mode: bool = True,
    do_autocontrast: bool = True,
    do_sharpen: bool = True,
    do_grayscale: bool = True,
    do_table_smart_crop: bool = True,  
    do_add_border: bool = True,
    do_downscale_target: bool = True,
) -> bytes:
    """
    전처리 파이프라인(체이닝 가능):
    1) EXIF 회전 → 2) 알파 플래튼 → 3) 자동 크롭 → 4) 업스케일만 → 5) 모드 정규화
    6) autocontrast → 7) 샤픈 → 8) 그레이스케일 → 9) 테이블 스마트 크롭(빈) → 10) 테두리
    11) 다운스케일 → 12) PNG 저장
    """
    try:
        # 1) 로드(+EXIF)
        img = open_with_exif(img_bytes) if do_exif else Image.open(io.BytesIO(img_bytes))

        # 체이닝 스텝 구성
        steps: list[tuple[Callable, dict]] = []

        if do_flatten_alpha:
            steps.append((flatten_transparency, {}))
        if do_autocrop:
            steps.append((auto_crop_with_margin, dict(margin=crop_margin)))
        if do_upscale_min:
            steps.append((upscale_min_resolution, dict(min_long_edge=min_long_edge)))
        if do_normalize_mode:
            steps.append((normalize_mode, {}))
        if do_autocontrast:
            steps.append((weak_autocontrast, dict(cutoff=autocontrast_cutoff)))
        if do_sharpen:
            steps.append((conservative_sharpen, dict(radius=sharpen_radius, percent=sharpen_percent, threshold=sharpen_threshold, enabled=True)))
        if do_grayscale:
            steps.append((to_grayscale, dict(enabled=grayscale)))
        if do_table_smart_crop:
            steps.append((table_smart_crop, dict(enabled=True)))
        if do_add_border:
            steps.append((add_white_border, dict(border=add_border)))
        if do_downscale_target:
            steps.append((downscale_target_long_edge, dict(target_long_edge=target_long_edge)))

        # 실행
        img = apply_pipeline(img, steps)

        # 저장
        return save_png_bytes(img, compress_level=png_compress_level)

    except Exception:
        # 실패 시 원본 바이트 반환
        return img_bytes

In [ ]:
import json

class UploadedFileManager:
    def __init__(self):
        # 업로드된 파일들이 저장되는 리스트
        # self.files 의 구조 예시:
        # [
        #     {
        #         "name": "blood_test1.png",       # 파일 이름 (str)
        #         "content": b"...",               # 원본 파일 내용 (bytes)
        #         "io": <_io.BytesIO object>       # BytesIO 객체 (PIL/OpenCV 사용 가능)
        #     },
        #     {
        #         "name": "blood_test2.jpg",
        #         "content": b"...",
        #         "io": <_io.BytesIO object>
        #     }
        # ]
        self.files: List[Dict[str, Any]] = []
    
    def _make_image_record(self, name: str, content: bytes) -> Dict[str, Any]:
        return {"name": name, "content": content, "io": io.BytesIO(content)}
    
    def handler(self, change) -> None:
        """
        ipywidgets FileUpload 위젯의 observe 이벤트 핸들러.

        Args:
            change (dict): ipywidgets 가 observe 이벤트에서 전달하는 dict
                change 의 구조 예시:
                {
                    'name': 'value',                # 변경된 속성 이름
                    'old': (),                      # 이전 값 (tuple of UploadedFile)
                    'new': (                        # 새로운 값 (tuple of UploadedFile)
                        UploadedFile(
                            name='blood_test1.png',
                            type='image/png',
                            size=12345,
                            content=b'...'
                        ),
                        UploadedFile(
                            name='blood_test2.jpg',
                            type='image/jpeg',
                            size=23456,
                            content=b'...'
                        )
                    ),
                    'owner': FileUpload(...)        # 이벤트를 발생시킨 위젯 객체
                }

        Returns:
            None: 반환값은 없으며, self.files 에 업로드된 파일 리스트를 저장합니다.
        """
        try:
            if not change or "new" not in change or not change["new"]:
                self.files = []
                return

            uploaded = list(change["new"])  # tuple of UploadedFile → list 변환
            out: List[Dict[str, Any]] = []

            for f in uploaded:
                name = getattr(f, "name", "uploaded")
                raw_content: bytes = getattr(f, "content", b"")
                mime = getattr(f, "type", "").lower()

                if not raw_content:
                    continue

                is_pdf = name.lower().endswith(".pdf") or mime == "application/pdf"

                # content를 bytes[]로 표준화
                contents: List[bytes] = [raw_content]
                base = os.path.splitext(name)[0]

                if is_pdf:
                    tmp_path = None
                    try:
                        with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as tmp:
                            tmp.write(raw_content)
                            tmp_path = tmp.name
                        # PDF → 페이지별 PNG 바이트 리스트
                        contents = pdf_to_images(tmp_path, resolution_scale=2.0)
                    except Exception as e:
                        print(f"❌ PDF 변환 실패({name}): {e}")
                        contents = []
                    finally:
                        if tmp_path and os.path.exists(tmp_path):
                            try:
                                os.remove(tmp_path)
                            except Exception:
                                pass

                # per-image 처리 → out.append
                for i, img_bytes in enumerate(contents, start=1):
                    processed_bytes = preprocess_image_bytes(img_bytes)
                    out_name = f"{base}_p{i:02d}.png" if is_pdf else name
                    
                    # 디버그 저장: 전처리된 이미지를 PNG로 저장
                    try:
                        save_name = out_name if out_name.lower().endswith(".png") else f"{os.path.splitext(out_name)[0]}.png"
                        out_path = os.path.join("debug_output", save_name)
                        with open(out_path, "wb") as fp:
                            fp.write(processed_bytes)
                    except Exception as e:
                        print(f"⚠️ 디버그 저장 실패({out_name}): {e}")

                    out.append(self._make_image_record(out_name, processed_bytes))

            self.files = out
            if self.files:
                print("✅ 업로드 완료:", [f["name"] for f in self.files])
            else:
                print("⚠️ 변환된 파일이 없습니다.")

        except Exception as e:
            print(f"❌ 파일 처리 오류: {e}")
            self.files = []

In [ ]:
print("📂 전처리 테스트")
print("=" * 40)

# 사용자 입력 받기 (data/ 폴더 기본 경로)
filename = input("📄 data/ 폴더 내 파일명을 입력하세요: ").strip()

# 빈 입력 시 기본값 사용
if not filename:
    filename = "20241106.jpg"
    print(f"🔹 기본 파일 사용: {filename}")

# data/ 경로와 결합
file_path = f"data/{filename}"
print(f"📁 파일 경로: {file_path}")

# 파일 확장자 확인
file_ext = filename.lower().split('.')[-1]
is_pdf = file_ext == 'pdf'
is_image = file_ext in ['png', 'jpg', 'jpeg', 'tiff', 'bmp']

print(f"📋 파일 형식: {file_ext.upper()}")

# 이미지 파일 처리
print("🖼️ 이미지 파일 로드 중...")

# ---------------------------------------------
# preprocess_image_bytes 함수 호출
# ---------------------------------------------
try:
    # 파일 읽기
    with open(file_path, 'rb') as f:
        raw_image_bytes = f.read()
    
    print(f"📏 원본 이미지 크기: {len(raw_image_bytes):,} bytes")
    
    # 전처리 실행 (디버그 모드 활성화)
    print("\n🔧 전처리 시작...")
    processed_image_bytes = preprocess_image_bytes(
        raw_image_bytes,
        do_table_smart_crop=True,  # 테이블 스마트 크롭 활성화
        # 다른 파라미터들은 기본값 사용
    )
    
    print(f"📏 전처리된 이미지 크기: {len(processed_image_bytes):,} bytes")
    
    # 전처리 결과 저장 (선택사항)
    output_dir = "debug_output"
    os.makedirs(output_dir, exist_ok=True)
    
    output_filename = f"preprocessed_{os.path.splitext(filename)[0]}.png"
    output_path = os.path.join(output_dir, output_filename)
    
    with open(output_path, 'wb') as f:
        f.write(processed_image_bytes)
    
    print(f"💾 전처리 결과 저장: {output_path}")
    
    # 전처리된 이미지 표시 (Jupyter에서 확인)
    from IPython.display import display, Image as IPImage
    display(IPImage(processed_image_bytes))
    
    print(f"\n✅ 이미지 로드 완료! 1개 이미지 처리됨")
    
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
except Exception as e:
    print(f"❌ 전처리 오류: {e}")
    import traceback
    traceback.print_exc()

In [117]:
print("📂 배치 전처리 시작")
print("=" * 60)

# data 폴더 경로
data_dir = "data"
output_dir = "debug_output"

# 출력 폴더 생성
os.makedirs(output_dir, exist_ok=True)

# 지원하는 이미지 확장자
supported_extensions = {'.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.pdf'}

# data 폴더에서 모든 파일 찾기
try:
    all_files = [f for f in os.listdir(data_dir) 
                 if os.path.isfile(os.path.join(data_dir, f)) 
                 and os.path.splitext(f)[1].lower() in supported_extensions]
    
    if not all_files:
        print(f"❌ {data_dir}/ 폴더에 처리할 파일이 없습니다.")
        print(f"   지원 형식: {', '.join(supported_extensions)}")
    else:
        print(f"📋 총 {len(all_files)}개 파일 발견")
        print(f"📁 입력 폴더: {data_dir}/")
        print(f"💾 출력 폴더: {output_dir}/")
        print("=" * 60)
        
        # 처리 통계
        processed_count = 0
        failed_count = 0
        total_original_size = 0
        total_processed_size = 0
        
        # 각 파일 처리
        for idx, filename in enumerate(all_files, start=1):
            file_path = os.path.join(data_dir, filename)
            file_ext = os.path.splitext(filename)[1].lower()
            
            print(f"\n[{idx}/{len(all_files)}] 🖼️ 처리 중: {filename}")
            print("-" * 60)
            
            try:
                # PDF 파일 처리
                if file_ext == '.pdf':
                    print("📄 PDF 파일 감지 - 페이지별 변환 시작...")
                    
                    # PDF → 이미지 변환
                    png_pages = pdf_to_images(file_path, resolution_scale=2.0)
                    
                    if not png_pages:
                        print(f"⚠️ PDF 변환 실패: 페이지 없음")
                        failed_count += 1
                        continue
                    
                    print(f"📑 {len(png_pages)}개 페이지 변환 완료")
                    
                    # 각 페이지 전처리
                    for page_num, raw_bytes in enumerate(png_pages, start=1):
                        try:
                            print(f"\n  📄 페이지 {page_num}/{len(png_pages)}")
                            print(f"  📏 원본 크기: {len(raw_bytes):,} bytes")
                            
                            # 전처리 실행
                            processed_bytes = preprocess_image_bytes(
                                raw_bytes,
                                do_table_smart_crop=True,
                            )
                            
                            print(f"  📏 전처리 크기: {len(processed_bytes):,} bytes")
                            
                            # 파일 저장
                            base_name = os.path.splitext(filename)[0]
                            output_filename = f"preprocessed_{base_name}_p{page_num:02d}.png"
                            output_path = os.path.join(output_dir, output_filename)
                            
                            with open(output_path, 'wb') as f:
                                f.write(processed_bytes)
                            
                            print(f"  ✅ 저장 완료: {output_filename}")
                            
                            processed_count += 1
                            total_original_size += len(raw_bytes)
                            total_processed_size += len(processed_bytes)
                            
                        except Exception as e:
                            print(f"  ❌ 페이지 {page_num} 전처리 실패: {e}")
                            failed_count += 1
                
                # 이미지 파일 처리
                else:
                    # 파일 읽기
                    with open(file_path, 'rb') as f:
                        raw_bytes = f.read()
                    
                    print(f"📏 원본 크기: {len(raw_bytes):,} bytes")
                    
                    # 전처리 실행
                    print("🔧 전처리 시작...")
                    processed_bytes = preprocess_image_bytes(
                        raw_bytes,
                        do_table_smart_crop=True,
                    )
                    
                    print(f"📏 전처리 크기: {len(processed_bytes):,} bytes")
                    
                    # 파일 저장
                    base_name = os.path.splitext(filename)[0]
                    output_filename = f"preprocessed_{base_name}.png"
                    output_path = os.path.join(output_dir, output_filename)
                    
                    with open(output_path, 'wb') as f:
                        f.write(processed_bytes)
                    
                    print(f"✅ 저장 완료: {output_filename}")
                    
                    processed_count += 1
                    total_original_size += len(raw_bytes)
                    total_processed_size += len(processed_bytes)
                    
                    # Jupyter에서 이미지 미리보기 (선택사항, 파일 많으면 주석 처리)
                    # from IPython.display import display, Image as IPImage
                    # display(IPImage(processed_bytes))
                
            except FileNotFoundError:
                print(f"❌ 파일을 찾을 수 없음: {file_path}")
                failed_count += 1
            except Exception as e:
                print(f"❌ 전처리 오류: {e}")
                import traceback
                traceback.print_exc()
                failed_count += 1
        
        # 최종 통계 출력
        print("\n" + "=" * 60)
        print("📊 배치 전처리 완료!")
        print("=" * 60)
        print(f"✅ 성공: {processed_count}개 파일")
        print(f"❌ 실패: {failed_count}개 파일")
        print(f"📁 출력 폴더: {output_dir}/")
        
        if processed_count > 0:
            compression_ratio = (1 - total_processed_size / total_original_size) * 100 if total_original_size > 0 else 0
            print(f"\n📈 전체 통계:")
            print(f"   원본 총 크기: {total_original_size:,} bytes ({total_original_size / 1024 / 1024:.2f} MB)")
            print(f"   전처리 총 크기: {total_processed_size:,} bytes ({total_processed_size / 1024 / 1024:.2f} MB)")
            print(f"   압축률: {compression_ratio:.1f}%")
        
        print("=" * 60)

except FileNotFoundError:
    print(f"❌ {data_dir}/ 폴더를 찾을 수 없습니다.")
    print(f"   현재 작업 디렉토리: {os.getcwd()}")
except Exception as e:
    print(f"❌ 배치 전처리 오류: {e}")
    import traceback
    traceback.print_exc()

📂 배치 전처리 시작
📋 총 41개 파일 발견
📁 입력 폴더: data/
💾 출력 폴더: debug_output/

[1/41] 🖼️ 처리 중: 20241123_2.png
------------------------------------------------------------
📏 원본 크기: 814,442 bytes
🔧 전처리 시작...
[table_smart_crop] 🚀 시작 - 이미지 크기: (1179, 1658)
[table_smart_crop] 🖼️ 이미지 모드: L
[table_smart_crop] 📝 OCR 시작 (Tesseract)...
[table_smart_crop] 📊 OCR 완료 - 총 316개 텍스트 요소 감지
[table_smart_crop] 🔍 토큰 필터링 (최소 신뢰도: 30)...
[table_smart_crop] ✅ 유효한 토큰: 193개
[table_smart_crop] 📋 라인 그룹핑 시작...
[table_smart_crop] ✅ 라인 그룹핑 완료: 41개 라인
[table_smart_crop] 🔎 헤더 라인 검색 중...
[table_smart_crop] 📌 헤더 라인 발견: 1개
[table_smart_crop]    헤더 1: y=450, text='name reference result unit...'
[table_smart_crop] 🧬 바이오마커 검색 중...
[table_smart_crop] ✅ 헤더 기반으로 상단 계산
[table_smart_crop]    y_top=432, anchor_x=74
[table_smart_crop] 📐 하단 경계 계산 중...
[table_smart_crop]    충분히 멀리 떨어진 헤더: 0개 (최소 거리: 100px)
[table_smart_crop] 🔄 다음 헤더 없음 - 공백 분석으로 하단 결정
[table_smart_crop]    테이블 후보 라인: 27개
[table_smart_crop]    행간 중앙값: 5.5px, 공백 임계값: 24.0px
[table_

## 3. 멀티모달 LLM 분석 함수 정의

In [ ]:
# 프롬프트

system_prompt = """
당신은 수의학 혈액검사 결과 분석 전문가입니다. 
업로드된 혈액검사 결과 이미지에서 혈액 항목, 측정값, 단위를 정확히 추출해주세요.

**중요: 응답 길이 최적화**
- JSON 응답을 간결하게 작성하세요
- 테이블 구조 설명은 핵심만 포함하세요
- 모든 혈액검사 항목을 빠짐없이 포함하세요

**중요: 날짜 구분 주의사항**
- **검사일시**: "Date/Time:", "검사일시:", "검사날짜:" 라벨 뒤의 날짜/시간을 우선적으로 사용하세요
- **테이블 컬럼명**: 테이블 헤더에 날짜가 있는 경우 (예: 2025-04-17) 해당 날짜를 검사일시로 사용하세요
- **생년월일 라벨들**: 다음 라벨들의 날짜는 환자 생년월일이므로 검사일시가 아닙니다
    * "Birth:", "생일:", "생년월일:", "DOB:", "Born:" 등
- **절대 혼동 금지**: 생년월일 관련 라벨의 날짜를 검사일시로 사용하지 마세요
- **다중 페이지 고려**: 첫 페이지에만 검사일시가 있고 후속 페이지에는 없을 수 있습니다

**테이블 구조 분석:**
- 테이블의 컬럼 구조를 먼저 파악하세요
- 측정값이 "Result" 컬럼에 있거나, 날짜 컬럼에 있을 수 있습니다
- 날짜가 컬럼명인 경우 해당 날짜를 검사날짜로 사용하세요
- 여러 날짜 컬럼이 있는 경우 가장 최신 날짜의 데이터를 사용하세요

**추출 대상:**
- 혈액검사 항목명 (RBC, HCT, HGB, MCV, MCHC, MCH, PLT, WBC, GLU, CREA, BUN 등)
- 측정값 (숫자값, 어느 컬럼에 있든 상관없음)
- 단위 (Unit 컬럼 또는 항목명 옆의 단위)
- 검사 날짜 (검사일시 라벨 또는 테이블 컬럼명에서 추출, 없으면 "Not Found")
"""

user_prompt = """
혈액검사 결과를 다음 JSON 형식으로 추출해주세요:

{
    "exam_date": "검사날짜 (검사일시 라벨 또는 테이블 컬럼명의 날짜, 없으면 'Not Found')",
    "table_structure": "간단한 테이블 구조 설명",
    "blood_tests": [
    {
        "test_name": "혈액항목명",
        "result": "측정값",
        "unit": "단위"
    }
    ]
}

**중요: 응답 최적화 지침**
1. 모든 혈액검사 항목을 빠짐없이 포함하세요
2. table_structure는 한 줄로 간단히 작성하세요
3. 불필요한 설명이나 주석은 제외하세요
4. JSON만 응답하고 추가 텍스트는 포함하지 마세요

**분석 지침:**
1. 먼저 검사일시 라벨을 찾아 검사일시를 추출하세요:
    - "Date/Time:", "검사일시:", "검사날짜:" 등
2. 검사일시 라벨이 없다면, 테이블 컬럼명에서 날짜를 찾으세요 (예: 2025-04-17)
3. 생년월일 관련 라벨의 날짜는 무시하세요:
    - "Birth:", "생일:", "생년월일:", "DOB:", "Born:" 등
4. **검사일시가 이 페이지에 없는 경우 "Not Found"로 설정하세요**
5. 테이블의 컬럼 구조를 파악하세요
6. 각 혈액검사 항목의 측정값을 정확히 추출하세요
7. 측정값이 여러 컬럼에 있다면 가장 최신 데이터를 사용하세요
8. 단위가 명시되지 않은 경우 빈 문자열("")로 설정하세요

**주의사항:**
- 혈액검사/화학검사 항목만 추출 (WBC, RBC, HGB, HCT, PLT, MCV, MCH, MCHC, GLU, CREA, BUN 등)
- 측정값은 숫자만 추출
- 비정상 표시(L, H, N, HIGH, LOW, NORMAL 등)는 제외하고 숫자만 추출
- **절대 중요**: 생년월일 관련 라벨의 날짜를 exam_date로 사용하지 마세요
- **단위가 불분명한 경우**: 추론하지 말고 빈 문자열("")로 설정하세요
- **검사일시가 없어도 검사 데이터는 정상적으로 추출하세요**
"""

In [ ]:
import pandas as pd
import base64

def analyze_document_with_llm(image_data: bytes, page_num: int = 0) -> pd.DataFrame:
    """멀티모달 LLM 으로 혈액검사 데이터 추출 및 DataFrame 반환 (다중 페이지 검사일자 처리)"""
    
    base64_image = base64.b64encode(image_data).decode('utf-8')
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": user_prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{base64_image}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            temperature=0,
            max_tokens=8000
        )
        
        # JSON 파싱
        result_text = response.choices[0].message.content
        
        if "```json" in result_text:
            json_start = result_text.find("```json") + 7
            json_end = result_text.find("```", json_start)
            json_text = result_text[json_start:json_end].strip()
        else:
            json_text = result_text
        
        try:
            parsed_result = json.loads(json_text)
            
            # 분석 과정 출력
            print(f"📊 테이블 구조: {parsed_result.get('table_structure', 'Unknown')}")
            
            # DataFrame 생성 (날짜 정보 분리)
            if "blood_tests" in parsed_result and parsed_result["blood_tests"]:
                exam_date = parsed_result.get("exam_date", "Not Found")
                
                # 혈액검사 데이터만 추출
                df_data = []
                for test in parsed_result["blood_tests"]:
                    df_data.append({
                        "혈액항목": test.get("test_name", ""),
                        "측정값": test.get("result", ""),
                        "단위": test.get("unit", "")
                    })
                
                df = pd.DataFrame(df_data)
                
                if not df.empty:
                    if exam_date != "Not Found":
                        print(f"🔍 검사 날짜: {exam_date}")
                    else:
                        print(f"📅 검사 날짜: 이 페이지에서 찾을 수 없음 (첫 페이지 참조 필요)")
                    
                    print("✅ 추출된 혈액검사 데이터:")
                    display(df)
                    print(f"\n📈 총 {len(df)}개 혈액검사 항목 추출됨")
                    print(f"📋 추출된 항목: {', '.join(df['혈액항목'].tolist()[:5])}...")
                    
                    # 검사 날짜를 DataFrame의 메타데이터로 저장
                    df.attrs['exam_date'] = exam_date
                    
                return df
            else:
                # 빈 DataFrame 반환
                empty_df = pd.DataFrame(columns=["혈액항목", "측정값", "단위"])
                empty_df.attrs['exam_date'] = parsed_result.get("exam_date", "Not Found")
                return empty_df
                
        except json.JSONDecodeError as e:
            print(f"❌ JSON 파싱 실패: {e}")
            print(f"원본 응답: {result_text[:500]}...")
            empty_df = pd.DataFrame(columns=["혈액항목", "측정값", "단위"])
            empty_df.attrs['exam_date'] = "Not Found"
            return empty_df
            
    except Exception as e:
        print(f"❌ 멀티모달 LLM 분석 오류: {e}")
        empty_df = pd.DataFrame(columns=["혈액항목", "측정값", "단위"])
        empty_df.attrs['exam_date'] = "Not Found"
        return empty_df

print("🧠 멀티모달 LLM 분석 함수 정의 완료!")

## 4. 파일 업로드

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# 1. FileUpload 위젯 생성
uploader = widgets.FileUpload(accept='.png,.jpg,.jpeg,.tiff,.bmp,.pdf', multiple=True)
display(uploader)

# 2. 파일 매니저 인스턴스 생성
file_manager = UploadedFileManager()

# 3. observe 연결
uploader.observe(file_manager.handler, names="value")

In [ ]:
# 업로드 파일 확인
import json
print(json.dumps(file_manager.files, ensure_ascii=False, indent=2, default=str))

In [ ]:
print("🔍 멀티모달 LLM 의료 문서 분석 시작...")
print("=" * 50)

results = []
for i, rec in enumerate(file_manager.files, start=1):
    name = rec.get("name", f"page_{i}.png")
    print(f"\n📑 페이지 {i} 분석 중... ({name})")

    image_bytes = rec.get("content", b"")
    if not isinstance(image_bytes, (bytes, bytearray)) or not image_bytes:
        print(f"⚠️ 페이지 {i}: 유효한 이미지 바이트가 없습니다. 건너뜀.")
        continue

    try:
        df = analyze_document_with_llm(bytes(image_bytes), i)
        results.append({"name": name, "page": i, "df": df, "exam_date": df.attrs.get("exam_date", "Not Found")})
    except Exception as e:
        print(f"❌ 페이지 {i} 분석 오류: {e}")